In [175]:
import warnings
import pandas as pd
import numpy as np
from datetime import date
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from matplotlib.patches import Patch
%matplotlib inline

## KONFIGURASI PANDAS DAN PLOT ##
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.option_context("display.max_rows", 100)
rcParams['figure.figsize'] = 12, 5
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("paper")
sns.set_palette("muted")

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.option_context("display.max_rows", 100)


pd.options.display.max_columns=999
pd.options.display.max_rows=999

import xgboost as xgb
import lime.lime_tabular
from datetime import date
from sklearn import tree
from sklearn import metrics
from collections import defaultdict
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, roc_curve, average_precision_score, precision_recall_curve


from scipy.stats import pearsonr, spearmanr
from scipy.stats import chi2_contingency
from scipy.stats import chi2

import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

In [176]:
df_train = pd.read_csv('20210303_dataset_agregasi_cleaned.csv')
df_train.head()

,cif_id,plafond_agg,purchase_amt_agg,loan_term_min_agg,loan_term_mean_agg,loan_term_max_agg,total_spent_l3d_agg,cumsum_bill_adj_agg,avg_credit_agg,pct_of_credit_agg,rule_1_all_agg,rule_2_all_agg,rule_3_all_agg,comb_rule_all_agg,pct_of_plf_pch_agg,pct_of_plf_l3d_agg,comb_rule_and_agg,cumul_comb_rule_all_agg,cumul_comb_rule_and_agg,cumul_avg_rule_all_agg,cumul_avg_rule_and_agg,freq_credit_last1m,rata_credit_last1m,freq_debit_last1m,rata_debit_last1m,freq_credit_last3m,rata_credit_last3m,freq_debit_last3m,rata_debit_last3m,freq_credit_last6m,rata_credit_last6m,freq_debit_last6m,rata_debit_last6m,age,collectibility,flag_payroll,income_now,income_apply,grade,label,avg_monthly_credit_last1m,avg_monthly_credit_last3m,avg_monthly_credit_last6m,avg_monthly_debit_last1m,avg_monthly_debit_last3m,avg_monthly_debit_last6m,avg_monthly_credit - debit_last1m,avg_monthly_credit - debit_last3m,avg_monthly_credit - debit_last6m,ratio_credit/debit_last1m,ratio_credit/debit_last3m,ratio_credit/debit_last6m,ratio_freq_credit/debit_last1m,ratio_freq_credit/debit_last3m,ratio_freq_credit/debit_last6m,umur_rekening (hari),umur_rekening (dalam tahun),min_credit_last_6m,max_credit_last_6m,min_debit_last_6m,max_debit_last_6m,min_credit/max_credit (last 6 month),interval_time,gender_Laki-laki,gender_Perempuan,marital_Duda/Janda,marital_Lajang,marital_Menikah,education_Diploma 1 (D1),education_Diploma 2 (D2),education_Diploma 3 (D3),education_Sekolah Dasar (SD),education_Sekolah Menengah Atas (SMA),education_Sekolah Menengah Pertama (SMP),education_Strata 1 (S1),education_Strata 2 (S2),education_Strata 3 (S3),occupation_Karyawan,occupation_Lain-Lain,occupation_Pensiunan,occupation_Profesional,occupation_TNI/POLRI,occupation_Wiraswasta
0,C000000004,5000000.0,3698553,3,4.500000,6,3698553,7.732015e+05,1.945309e+08,0.008286,1,0,0,1,0.739711,0.739711,1,1,1,0.500000,0.500000,9,6621546.993,119,1.341754e+06,31,8658323.878,282,9.252740e+05,73,7.420370e+06,564,9.581919e+05,39.0,1,0,26821117.5,20000000.0,1.0,0,59593922.94,89469346.74,9.028117e+07,159668775.0,86975751.93,9.007004e+07,-100074852.1,2.493595e+06,2.111313e+05,4.934992,9.357579,7.744138,0.075630,0.109929,0.129433,4556,12.482192,59593922.94,140135500.3,36606717.0,159668775.0,0.425259,13.929861,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,C000000001,5000000.0,8404849,1,2.312500,6,9953251,1.140943e+07,1.575923e+09,0.237128,0,0,0,0,1.680970,1.990650,0,0,0,0.000000,0.000000,6,2334304.167,69,8.319019e+05,17,4663068.571,146,5.220592e+05,33,3.684057e+06,266,4.420984e+05,28.0,1,0,11897296.0,10000000.0,1.0,0,14005825.00,26424055.23,2.026232e+07,57401229.4,25406883.13,1.959970e+07,-43395404.4,1.017172e+06,6.626184e+05,2.805985,8.932068,8.333116,0.086957,0.116438,0.124060,2337,6.402740,10523068.00,52208401.7,7049339.0,57401229.4,0.201559,13.931944,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,C000000015,5000000.0,9441712,3,3.000000,3,9441712,7.494278e+06,4.254102e+08,0.216590,1,0,0,1,1.888342,1.888342,1,4,4,0.500000,0.500000,6,2527438.500,39,1.130489e+06,31,6516086.219,123,1.459606e+06,59,1.226898e+07,316,2.471159e+06,29.0,1,0,13184640.5,14000000.0,1.0,0,15164631.00,67332890.93,1.206449e+08,44089089.0,59843825.67,1.301477e+08,-28924458.0,7.489065e+06,-9.502764e+06,2.235703,4.464279,4.964868,0.153846,0.252033,0.186709,2341,6.413699,15164631.00,442602266.0,34338583.0,538037997.0,0.034262,8.933333,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,C000000037,5000000.0,6336171,1,5.714286,12,7422671,2.358537e+06,1.467465e+08,0.154450,1,1,0,1,1.267234,1.484534,2,6,12,0.857143,1.714286,14,1053968.000,45,2.421133e+05,26,1371942.308,130,2.504312e+05,50,1.316269e+06,264,2.498908e+05,26.0,1,0,8629410.0,8000000.0,1.0,0,14755552.00,11890166.67,1.096891e+07,10895100.0,10852017.33,1.099519e+07,3860452.0,1.038149e+06,-2.628717e+04,4.353201,5.478321,5.267377,0.311111,0.200000,0.189394,516,1.413699,8082390.00,14755552.0,7858583.0,12469629.0,0.547752,9.128472,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
4,C000000068,5000000.0,5950931,3,3.000000,3,7020176,2.

In [177]:
#df_int = df_train.select_dtypes(exclude=['object'])
df_int = df_train

In [178]:
list_column = df_int.columns.to_list()
list_inf    = []

for i in range(df_int.shape[1]):
    list_inf.append(df_int[list_column[i]].max())
    
inf_list = pd.DataFrame(
    {'Fitur': list_column,
     'Inf ?': list_inf,
    })

df_int.shape

(10147, 83)

# Drop Inf Value

In [179]:
df_feature = df_int.replace([np.inf, -np.inf], np.nan).dropna()

# Remove Highly Correlated Features

In [6]:
# Create correlation matrix
corr_matrix = df_feature.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.975)]

print(f'There are {len(to_drop)} correlated columns to remove.')
print(to_drop)

There are 11 correlated columns to remove.
['comb_rule_and_agg', 'cumul_comb_rule_and_agg', 'rata_debit_last6m', 'avg_monthly_credit_last6m', 'avg_monthly_debit_last1m', 'avg_monthly_debit_last3m', 'avg_monthly_debit_last6m', 'umur_rekening (dalam tahun)', 'min_debit_last_6m', 'max_debit_last_6m', 'gender_Perempuan']


In [128]:
dataset_new = df_feature.drop(columns = to_drop)

KeyError: "['credit.loan_term_mean_agg' 'credit.COUNT(ceria)'\n 'credit.MAX(ceria.avg_monthly_credit - debit_last3m)'\n 'credit.MAX(ceria.avg_monthly_credit - debit_last6m)'\n 'credit.MAX(ceria.avg_monthly_credit_last1m)'\n 'credit.MAX(ceria.avg_monthly_credit_last3m)'\n 'credit.MAX(ceria.avg_monthly_credit_last6m)'\n 'credit.MAX(ceria.avg_monthly_debit_last1m)'\n 'credit.MAX(ceria.avg_monthly_debit_last3m)'\n 'credit.MAX(ceria.avg_monthly_debit_last6m)'\n 'credit.MAX(ceria.comb_rule_and_agg)'\n 'credit.MAX(ceria.cumul_avg_rule_all_agg)'\n 'credit.MAX(ceria.cumul_avg_rule_and_agg)'\n 'credit.MAX(ceria.cumul_comb_rule_all_agg)'\n 'credit.MAX(ceria.cumul_comb_rule_and_agg)'\n 'credit.MAX(ceria.education_Sekolah Dasar (SD))'\n 'credit.MAX(ceria.freq_credit_last3m)'\n 'credit.MAX(ceria.freq_credit_last6m)'\n 'credit.MAX(ceria.freq_debit_last1m)'\n 'credit.MAX(ceria.freq_debit_last3m)'\n 'credit.MAX(ceria.freq_debit_last6m)'\n 'credit.MAX(ceria.max_credit_last_6m)'\n 'credit.MAX(ceria.max_d

In [192]:
x = pd.DataFrame(df_feature['label'])
x = x.reset_index()
x.drop(['index'], axis=1, inplace=True)

In [204]:
x['label']

0       0
1       0
2       0
3       0
4       0
       ..
9864    0
9865    0
9866    0
9867    0
9868    0
Name: label, Length: 9869, dtype: int64

In [195]:
# saving identifiers
test_cif_Identifier = df_feature['cif_id']
#test_Outlet_Identifier = test['Outlet_Identifier']
sales = df_feature['label']
df_feature.drop(['label'], axis=1, inplace=True)
df_feature.drop(['collectibility'], axis=1, inplace=True)

In [196]:
sales

0        0
1        0
2        0
3        0
4        0
        ..
10142    0
10143    0
10144    0
10145    0
10146    0
Name: label, Length: 9869, dtype: int64

In [197]:
# creating and entity set 'es'
es = ft.EntitySet(id = 'sales')

# adding a dataframe 
es.entity_from_dataframe(entity_id = 'ceria', dataframe = df_feature, index = 'cif_id')

Entityset: sales
  Entities:
    ceria [Rows: 9869, Columns: 81]
  Relationships:
    No relationships

In [198]:
es.normalize_entity(base_entity_id='ceria', new_entity_id='credit', index = 'loan_term_min_agg', 
                    additional_variables = ['loan_term_mean_agg', 'loan_term_max_agg', 'total_spent_l3d_agg',
       'cumsum_bill_adj_agg', 'avg_credit_agg', 'pct_of_credit_agg',])

Entityset: sales
  Entities:
    ceria [Rows: 9869, Columns: 75]
    credit [Rows: 5, Columns: 7]
  Relationships:
    ceria.loan_term_min_agg -> credit.loan_term_min_agg

In [199]:
print(es)

Entityset: sales
  Entities:
    ceria [Rows: 9869, Columns: 75]
    credit [Rows: 5, Columns: 7]
  Relationships:
    ceria.loan_term_min_agg -> credit.loan_term_min_agg


In [200]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'ceria', 
max_depth = 2, 
verbose = 1, 
n_jobs = 3)

Built 519 features
EntitySet scattered to 3 workers in 3 seconds                                                                          
Elapsed: 00:03 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [201]:
feature_matrix.head()

,plafond_agg,purchase_amt_agg,loan_term_min_agg,rule_1_all_agg,rule_2_all_agg,rule_3_all_agg,comb_rule_all_agg,pct_of_plf_pch_agg,pct_of_plf_l3d_agg,comb_rule_and_agg,cumul_comb_rule_all_agg,cumul_comb_rule_and_agg,cumul_avg_rule_all_agg,cumul_avg_rule_and_agg,freq_credit_last1m,rata_credit_last1m,freq_debit_last1m,rata_debit_last1m,freq_credit_last3m,rata_credit_last3m,freq_debit_last3m,rata_debit_last3m,freq_credit_last6m,rata_credit_last6m,freq_debit_last6m,rata_debit_last6m,age,flag_payroll,income_now,income_apply,grade,avg_monthly_credit_last1m,avg_monthly_credit_last3m,avg_monthly_credit_last6m,avg_monthly_debit_last1m,avg_monthly_debit_last3m,avg_monthly_debit_last6m,avg_monthly_credit - debit_last1m,avg_monthly_credit - debit_last3m,avg_monthly_credit - debit_last6m,ratio_credit/debit_last1m,ratio_credit/debit_last3m,ratio_credit/debit_last6m,ratio_freq_credit/debit_last1m,ratio_freq_credit/debit_last3m,ratio_freq_credit/debit_last6m,umur_rekening (hari),umur_rekening (dalam tahun),min_credit_last_6m,max_credit_last_6m,min_debit_last_6m,max_debit_last_6m,min_credit/max_credit (last 6 month),interval_time,gender_Laki-laki,gender_Perempuan,marital_Duda/Janda,marital_Lajang,marital_Menikah,education_Diploma 1 (D1),education_Diploma 2 (D2),education_Diploma 3 (D3),education_Sekolah Dasar (SD),education_Sekolah Menengah Atas (SMA),education_Sekolah Menengah Pertama (SMP),education_Strata 1 (S1),education_Strata 2 (S2),education_Strata 3 (S3),occupation_Karyawan,occupation_Lain-Lain,occupation_Pensiunan,occupation_Profesional,occupation_TNI/POLRI,occupation_Wiraswasta,credit.loan_term_mean_agg,credit.loan_term_max_agg,credit.total_spent_l3d_agg,credit.cumsum_bill_adj_agg,credit.avg_credit_agg,credit.pct_of_credit_agg,credit.COUNT(ceria),credit.MAX(ceria.age),credit.MAX(ceria.avg_monthly_credit - debit_last1m),credit.MAX(ceria.avg_monthly_credit - debit_last3m),credit.MAX(ceria.avg_monthly_credit - debit_last6m),credit.MAX(ceria.avg_monthly_credit_last1m),credit.MAX(ceria.avg_monthly_credit_last3m),credit.MAX(ceria.avg_monthly_credit_last6m),credit.MAX(ceria.avg_monthly_debit_last1m),credit.MAX(ceria.avg_monthly_debit_last3m),credit.MAX(ceria.avg_monthly_debit_last6m),credit.MAX(ceria.comb_rule_all_agg),credit.MAX(ceria.comb_rule_and_agg),credit.MAX(ceria.cumul_avg_rule_all_agg),credit.MAX(ceria.cumul_avg_rule_and_agg),credit.MAX(ceria.cumul_comb_rule_all_agg),credit.MAX(ceria.cumul_comb_rule_and_agg),credit.MAX(ceria.education_Diploma 1 (D1)),credit.MAX(ceria.education_Diploma 2 (D2)),credit.MAX(ceria.education_Diploma 3 (D3)),credit.MAX(ceria.education_Sekolah Dasar (SD)),credit.MAX(ceria.education_Sekolah Menengah Atas (SMA)),credit.MAX(ceria.education_Sekolah Menengah Pertama (SMP)),credit.MAX(ceria.education_Strata 1 (S1)),credit.MAX(ceria.education_Strata 2 (S2)),credit.MAX(ceria.education_Strata 3 (S3)),credit.MAX(ceria.flag_payroll),credit.MAX(ceria.freq_credit_last1m),credit.MAX(ceria.freq_credit_last3m),credit.MAX(ceria.freq_credit_last6m),credit.MAX(ceria.freq_debit_last1m),credit.MAX(ceria.freq_debit_last3m),credit.MAX(ceria.freq_debit_last6m),credit.MAX(ceria.gender_Laki-laki),credit.MAX(ceria.gender_Perempuan),credit.MAX(ceria.grade),credit.MAX(ceria.income_apply),credit.MAX(ceria.income_now),credit.MAX(ceria.interval_time),credit.MAX(ceria.marital_Duda/Janda),credit.MAX(ceria.marital_Lajang),credit.MAX(ceria.marital_Menikah),credit.MAX(ceria.max_credit_last_6m),credit.MAX(ceria.max_debit_last_6m),credit.MAX(ceria.min_credit/max_credit (last 6 month)),credit.MAX(ceria.min_credit_last_6m),credit.MAX(ceria.min_debit_last_6m),credit.MAX(ceria.occupation_Karyawan),credit.MAX(ceria.occupation_Lain-Lain),credit.MAX(ceria.occupation_Pensiunan),credit.MAX(ceria.occupation_Profesional),credit.MAX(ceria.occupation_TNI/POLRI),credit.MAX(ceria.occupation_Wiraswasta),credit.MAX(ceria.pct_of_plf_l3d_agg),credit.MAX(ceria.pct_of_plf_pch_agg),credit.MAX(ceria.plafond_agg),credit.MAX(ceria.purchase_amt_agg),credit.MAX(ceria.rata

In [202]:
feature_matrix = feature_matrix.reindex(index=df_feature['cif_id'])
feature_matrix = feature_matrix.reset_index()

In [205]:
dataset_ftool = pd.concat([feature_matrix, x['label']],axis=1)

In [211]:
dataset_ftool.shape

(9869, 521)

In [218]:
dataset_ftool.to_csv("%s_dataset_agg_ftool_before_corr.csv" % str(date.today().strftime("%Y%m%d")), index=False)

# Saving Dataset sebelum correlation antar fitur

In [212]:
# %%
## SPLIT DATASET INTO DATA TRAIN AND DATA TEST
data_train, data_test = train_test_split(dataset_ftool, test_size=0.30, random_state=24)

data_train.to_csv("%s_data_train_agg_ftool_before_corr.csv" % str(date.today().strftime("%Y%m%d")), index=False)
data_test.to_csv("%s_data_test_agg_ftool_before_corr.csv" % str(date.today().strftime("%Y%m%d")), index=False)

In [216]:
# Create correlation matrix
corr_matrix = dataset_ftool.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.9)]

print(f'There are {len(to_drop)} correlated columns to remove.')
print(to_drop)

There are 388 correlated columns to remove.
['comb_rule_all_agg', 'comb_rule_and_agg', 'cumul_comb_rule_and_agg', 'cumul_avg_rule_all_agg', 'cumul_avg_rule_and_agg', 'freq_credit_last3m', 'rata_debit_last3m', 'freq_credit_last6m', 'freq_debit_last6m', 'rata_debit_last6m', 'avg_monthly_credit_last3m', 'avg_monthly_credit_last6m', 'avg_monthly_debit_last1m', 'avg_monthly_debit_last3m', 'avg_monthly_debit_last6m', 'ratio_freq_credit/debit_last6m', 'umur_rekening (dalam tahun)', 'min_credit_last_6m', 'max_credit_last_6m', 'min_debit_last_6m', 'max_debit_last_6m', 'gender_Perempuan', 'marital_Menikah', 'credit.loan_term_mean_agg', 'credit.loan_term_max_agg', 'credit.avg_credit_agg', 'credit.pct_of_credit_agg', 'credit.COUNT(ceria)', 'credit.MAX(ceria.age)', 'credit.MAX(ceria.avg_monthly_credit - debit_last3m)', 'credit.MAX(ceria.avg_monthly_credit - debit_last6m)', 'credit.MAX(ceria.avg_monthly_credit_last1m)', 'credit.MAX(ceria.avg_monthly_credit_last3m)', 'credit.MAX(ceria.avg_monthly_cre

In [37]:
dataset_after_corr = feature_matrix.drop(columns = to_drop)